In [1]:
using RigidBodyDynamics
using StaticArrays
using RigidBodyDynamics.TreeDataStructure
import RigidBodyDynamics: VectorSegment
import RigidBodyDynamics: num_positions, num_velocities
using BenchmarkTools

In [2]:
const ScalarType = Float64
# const ScalarType = Float32

function create_floating_atlas()
    atlasUrdfUrl = "https://raw.githubusercontent.com/RobotLocomotion/drake/6e3ca768cbaabf15d0f2bed0fb5bd703fa022aa5/drake/examples/Atlas/urdf/atlas_minimal_contact.urdf"
    atlasUrdf = RigidBodyDynamics.cached_download(atlasUrdfUrl, "atlas.urdf")
    atlas = parse_urdf(ScalarType, atlasUrdf)
#     for child in children(root_vertex(atlas))
#         joint = edge_to_parent_data(child)
#         change_joint_type!(atlas, joint, QuaternionFloating{ScalarType}())
#     end
    atlas
end

create_floating_atlas (generic function with 1 method)

In [5]:
const mechanism = create_floating_atlas()
remove_fixed_joints!(mechanism);

In [3]:
include(joinpath(Pkg.dir("RigidBodyDynamics"), "src", "codegen.jl"))
using RBDCodeGen

In [6]:
const state = RBDCodeGen.MechanismState(Float64, mechanism);

In [7]:
# @benchmark t1 * t2 setup = (element = state.elements[4]; t1 = element.beforeJointToParent; j = element.joint; q = element.q; t2 = joint_transform(j, q))
# @benchmark X1 * X2 setup = (X1 = rand(SMatrix{4, 4}); X2 = rand(SMatrix{4, 4}))
# @benchmark (R1 * R2, R1 * p2 + p1) setup = (R1 = rand(SMatrix{3, 3}); R2 = rand(SMatrix{3, 3}); p1 = rand(SVector{3}); p2 = rand(SVector{3}))

In [8]:
nv = num_velocities(mechanism)
nq = num_positions(mechanism)
const M = Symmetric(Matrix{Float64}(nv, nv), :L);

In [9]:
@benchmark update_joint_transforms($state) setup = (rand!(state.q); rand!(state.v))

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.668 μs (0.00% GC)
  median time:      2.794 μs (0.00% GC)
  mean time:        2.843 μs (0.00% GC)
  maximum time:     13.284 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [11]:
@benchmark update_crb_inertias($state) setup = (rand!(state.q); rand!(state.v))

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.074 μs (0.00% GC)
  median time:      2.148 μs (0.00% GC)
  mean time:        2.322 μs (0.00% GC)
  maximum time:     10.789 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [10]:
@time mass_matrix!(M, state)

  1.106989 seconds (660.34 k allocations: 25.622 MB, 1.08% gc time)


In [12]:
@benchmark mass_matrix!($M, $state) setup = (rand!(state.q); rand!(state.v))

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     19.451 μs (0.00% GC)
  median time:      20.872 μs (0.00% GC)
  mean time:        23.155 μs (0.00% GC)
  maximum time:     126.257 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [13]:
# test(n) = for i = 1 : n mass_matrix!(M, state) end
# using ProfileView
# test(1)
# Profile.clear()
# @profile test(100000)
# ProfileView.view()

In [14]:
using ForwardDiff

In [15]:
const chunk_size = 10

10

In [16]:
nv = num_velocities(mechanism)
const state_autodiff = RBDCodeGen.MechanismState(ForwardDiff.Dual{chunk_size, Float64}, mechanism);
const M_autodiff = Symmetric(Matrix{ForwardDiff.Dual{chunk_size, Float64}}(nv, nv), :U);

In [17]:
function f!(out, q)
    copy!(state_autodiff.q, q)
    mass_matrix!(M_autodiff, state_autodiff)
    @simd for i in eachindex(out)
        out[i] = M_autodiff[i]
    end
end

f! (generic function with 1 method)

In [18]:
const out = Matrix{Float64}(nv * nv, nq)
const y = Vector{Float64}(nv * nv)
@time ForwardDiff.jacobian!(out, f!, y, zeros(nv), ForwardDiff.JacobianConfig{chunk_size}(y, zeros(nq)));

216.093065 seconds (201.55 M allocations: 3.117 GB, 0.30% gc time)


In [15]:
@benchmark ForwardDiff.jacobian!($out, $f!, $y, q, config) setup = (q = rand(num_positions(mechanism)); config = ForwardDiff.JacobianConfig{chunk_size}(y, q))

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     898.840 μs (0.00% GC)
  median time:      939.674 μs (0.00% GC)
  mean time:        990.237 μs (0.00% GC)
  maximum time:     1.688 ms (0.00% GC)
  --------------
  samples:          4972
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [19]:
const q = rand(num_velocities(mechanism));
const config = ForwardDiff.JacobianConfig{chunk_size}(y, q)
test2(n) = for i = 1 : n ForwardDiff.jacobian!(out, f!, y, q, config) end

test2 (generic function with 1 method)

In [2]:
# using ProfileView
# test2(1)
# Profile.clear()
# @profile test2(10000)
# ProfileView.view()